# Overview

**GENERAL THOUGHTS:**  
Use PyCaret (pycaret.classification) as a general way to investigate which algorithm, with automated pre-processing are (well) suited for the given tasks, as well as to investigate the potential performance based on a (large) varity of model hyper-parameters.
The notebook includes multiple scenarios of using PyCaret:
- including and excluding custom data pre-processing (see below)
- including auto pre-processing by PyCaret
- including multiple classifiers by using:
  - multiple ml algorithms with a base configuration of their hyper-parameters defined within PyCaret
  - "standard" HPO for each algorithm with a defined search space by PyCaret and random search as search strategy with 15 random hyper-parameter configurations for each algorithm. https://github.com/pycaret/pycaret/blob/master/pycaret/containers/models/classification.py

**CUSTOM DATA PREPROCESSING:**

Imbalanced data:
- over_sampling for imbalanced data
- cost-sensitive learning for imbalanced data

**PyCaret MULTI-CLASS CLASSIFIERS:**
Class weights are not considered during training when using `compare_models` or `tune_models`. As an evaluation metric `f1_macro` was choosen, which equally consideres all classes. Since training is not optimizied regarding this aspect, results could be improved for training individual models with 'create_model` which supports class weights. For comparison we neglect class weights for the reason of an easy use of PyCaret. The effect of considering class weights can vary highly depeding on the machine learning algorithm (e.g. splits in decision trees, distance calculation in KNN).
- Overview of models to be considered using PyCaret:  
  - [X] RandomForest
  - [X] ExtraTrees
  - [X] XGBoost
  - [X] LightGBM
  - [X] KNeughbors
  - [X] CatBoost
  - [X] Decision Tree Classifier
  - [X] Gradient Boosting Classifier
  - [X] Extreme Gradient Boosting
  - [X] catboost	CatBoost Classifier
  - [X] Extra Trees Classifier
  - [X] Random Forest Classifier
  - [X] K Neighbors Classifier
  - [X] Linear Discriminant Analysis
  - [X] Ridge Classifier
  - [X] Naive Bayes
  - [X] Quadratic Discriminant Analysis
  - [X] Ada Boost Classifier
  - [X] Light Gradient Boosting Machine
  - [X] Logistic Regression
  - [X] SVM - Linear Kernel
  - [X] Dummy Classifier

**FINAL MODEL PERFORMANCE:**  
- Evaluation of the best model from AutoML, including Experiment checkpointing.
- Loading final model from checkpoint for prediction on test set for evaluation based on classification report
- Tracking of the best model with MLFlow for performance benchmarking with other approaches (Baseline, PyCaret, AutoGluon, PyTorch, ...) within the Repository.

In [1]:
colab = False

In [2]:
if colab:
  # Import the library to mount Google Drive
  from google.colab import drive
  # Mount the Google Drive at /content/drive
  drive.mount('/content/drive')
  # Verify by listing the files in the drive
  # !ls /content/drive/My\ Drive/
  # current dir in colab
  !pwd

In [3]:
if colab:
    !pip install --upgrade optuna==3.5.0
    # !pip install --upgrade optuna.integration
    !pip install --upgrade mlflow
    !pip install --upgrade PyCaret

In [4]:
import os
import sys
import yaml
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import classification_report, f1_score
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_sample_weight
import imblearn
from imblearn.over_sampling import RandomOverSampler

import pycaret
# import ClassificationExperiment
from pycaret.classification import *
# from pycaret.classification import ClassificationExperiment

import mlflow

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# check installed version
pycaret.__version__

'3.3.1'

In [6]:
# NOTE: if used in google colab, upload env_vars_colab.yml to current google colab directory!

# get config
if colab:
    with open('./env_vars_colab.yml', 'r') as file:
        config = yaml.safe_load(file)
else:
    with open('../env_vars_azureml_compute.yml', 'r') as file:
        config = yaml.safe_load(file)

# custom imports
sys.path.append(config['project_directory'])

from src import utils

In [7]:
SEED = 42

In [8]:
# General settings within the data science workflow

pd.set_option('display.max_columns', None)

# NOTE: for dev only
subsample = False
subsample_size = 100  # subsample subset of data for faster demo or development

# Get current date and time
now = datetime.datetime.now()
# Format date and time
formatted_date_time = now.strftime("%Y-%m-%d_%H:%M:%S")
print(formatted_date_time)

2024-11-14_19:32:32


# Load and prepare data

In [9]:
df = pd.read_csv(f"{config['data_directory']}/output/df_ml.csv", sep='\t')

df['material_number'] = df['material_number'].astype('object')

df_sub = df[[
    'material_number',
    'brand',
    'product_area',
    'core_segment',
    'component',
    'manufactoring_location',
    'characteristic_value',
    'material_weight',
    'packaging_code',
    'packaging_category',
]]

## Transform to PyCaret data format
When you execute the setup function in PyCaret it splits the data into train and test sets (70/30) by default. Cross-validation is then done on train set only.
The hold-out set is there just for an additional sense of surety.

In [10]:
df_sub.head()

,material_number,brand,product_area,core_segment,component,manufactoring_location,characteristic_value,material_weight,packaging_code,packaging_category
0,75116293,BOT,PA5,Metal Grinding,6035765C21,Distribution Center,CORRUGATED,85.00,PCode_304109,Countertop display
1,75116293,BOT,PA5,Metal Grinding,6035940565,Distribution Center,WOOD FREE,0.54,PCode_440854,Countertop display
2,75116293,BOT,PA5,Metal Grinding,6035822768,Distribution Center,MCB/GT2,22.90,PCode_834649,Countertop display
3,75116293,BOT,PA5,Metal Grinding,6035822768,Distribution Center,MCB/GT2,22.90,PCode_834649,Countertop display
4,75116293,BOT,PA5,Metal Grinding,6035765P54,Distribution Center,CORRUGATED,85.00,PCode_304109,Countertop display


# PyCaret AutoML: without custom pre-processing; unrestricted selection of models including HPO

## PyCaret Base Models Training Pipeline

In [11]:
# init ClassificationExperiment
exp_base = ClassificationExperiment()

print(f"Experiment Type: {type(exp_base)}") # check the type of exp

Experiment Type: <class 'pycaret.classification.oop.ClassificationExperiment'>


In [12]:
# init setup on experiment
exp_base.setup(
    df_sub,
    target='packaging_category',
    train_size=0.8,
    fold=5,
    fold_strategy='stratifiedkfold',
    session_id=42
)

,Description,Value
0,Session id,42
1,Target,packaging_category
2,Target type,Multiclass
3,Target mapping,"Blister and Insert Card: 0, Blister and sealed blist: 1, Book packaging: 2, Cardb. Sleeve w - w/o Shr.: 3, Cardboard hanger w/o bag: 4, Carton cover (Lid box): 5, Carton tube with or w/o: 6, Case: 7, Corrugated carton: 8, Countertop display: 9, Envelope: 10, Fabric packaging: 11, Folding carton: 12, Hanger/ Clip: 13, Metal Cassette: 14, Paperboard pouch: 15, Plastic Box: 16, Plastic Cassette: 17, Plastic Pouch: 18, Plastic bag with header: 19, Shrink film and insert o: 20, Skincard: 21, TightPack: 22, Trap Card: 23, Trap Folding Card: 24, Tray Packer: 25, Tube: 26, Unpacked: 27, Wooden box: 28"
4,Original data shape,"(82977, 10)"
5,Transformed data shape,"(82977, 66)"
6,Transformed train set shape,"(66381, 66)"
7,Transformed test set shape,"(16596, 66)"
8,Numeric features,1
9,Categorical features,8


In [13]:
# add sklearn f1_score macro average
exp_base.add_metric(id='f1_macro', name='F1_Macro', score_func=utils.f1_score_macro, greater_is_better=True)
exp_base.remove_metric('MCC')
exp_base.remove_metric('Kappa')
exp_base.get_metrics()

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x7f5cada02ca0>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(roc_auc_score, response_method=('d...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, response_method='pre...",pred,{'average': 'weighted'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, response_method='...",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, response_method='predict...",pred,{'average': 'weighted'},True,True,False
f1_macro,F1_Macro,F1_Macro,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(f1_score_macro, response_method='p...",pred,{},True,True,True


In [14]:
# train and compare base models

# #NOTE: class_weights are not yet supported in compare_models
# class_weights = class_weight.compute_class_weight(
#     class_weight="balanced",
#     classes=np.unique(df_sub.iloc[:, -1]),
#     y=df_sub.iloc[:, -1]
# )
# class_weight_dict = dict(enumerate(class_weights))
# basemodels = exp_base.compare_models(include=['dt', 'rf'], sort='F1_Macro', fit_kwargs={'class_weight': class_weight_dict})
# lr_clf = exp_base.create_model('lr', class_weight=class_weight_dict)

base_models = exp_base.compare_models(sort='F1_Macro', n_select=exp_base.models().shape[0])

,Model,Accuracy,AUC,Recall,Prec.,F1,F1_Macro,TT (Sec)
dt,Decision Tree Classifier,0.8528,0.0000,0.8528,0.9870,0.9111,0.8185,1.8000
xgboost,Extreme Gradient Boosting,0.8830,0.0000,0.8830,0.9745,0.9198,0.8121,31.8120
gbc,Gradient Boosting Classifier,0.8485,0.0000,0.8485,0.9486,0.8802,0.8026,283.7560
catboost,CatBoost Classifier,0.8853,0.0000,0.8853,0.9406,0.8987,0.7911,265.5540
et,Extra Trees Classifier,0.9213,0.0000,0.9213,0.9346,0.9236,0.7796,6.0820
rf,Random Forest Classifier,0.9117,0.0000,0.9117,0.9273,0.9141,0.7601,5.7000
knn,K Neighbors Classifier,0.8941,0.0000,0.8941,0.8996,0.8929,0.6881,9.8600
lda,Linear Discriminant Analysis,0.7422,0.0000,0.7422,0.8029,0.7547,0.5064,2.6380
ridge,Ridge Classifier,0.6213,0.0000,0.6213,0.5820,0.5643,0.2456,1.6840
nb,Naive Bayes,0.4711,0.0000,0.4711,0.5113,0.4184,0.2076,1.8440


In [16]:
# Create the directory if it doesn't exist
os.makedirs(config['pycaret_exp_storage_directory'], exist_ok=True)
exp_filename = "exp_base.pkl"
exp_base.save_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")

## PyCaret Best Model Evaluation

In [17]:
try:
    exp_filename = "exp_base.pkl"
    try:
        # Check if exp_base is defined and assign it only if it exists. "or" statment in case exp_base is falsy.
        exp_base = exp_base or ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
    except NameError:
        # If exp_base is not defined, assign it by loading the experiment.
        exp_base = ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
        print("Loading experiment from storage.")
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"The experiment {exp_filename} does not exist and could not be loaded.")

In [ ]:
leaderboard_base = exp_base.get_leaderboard()
leaderboard_base.sort_values(by='F1_Macro', ascending=False)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,22:11:59
Status,. . . . . . . . . . . . . . . . . .,Collecting models
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


Processing:   0%|          | 0/17 [00:00<?, ?it/s]

## PyCaret Tuned Models Training Pipeline

In [ ]:
try:
    exp_filename = "exp_base.pkl"
    try:
        # Check if exp_tuned is defined and assign it only if it exists. "or" statment in case exp_base is falsy.
        exp_tuned = exp_base or ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
        print("Experiment loaded from storage.")
    except NameError:
        # If exp_tuned is not defined, assign it by loading the experiment.
        exp_tuned = ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"The experiment {exp_filename} does not exist and could not be loaded.")
    print("Define new pycaret experiment for tuning.")
    exp_tuned = ClassificationExperiment()
    exp_tuned.setup(
        df_sub,
        target='packaging_category',
        train_size=0.8,
        fold=5,
        fold_strategy='stratifiedkfold',
        session_id=42
    )
    # add sklearn f1_score macro average
    exp_tuned.add_metric(id='f1_macro', name='F1_Macro', score_func=utils.f1_score_macro, greater_is_better=True)
    exp_tuned.remove_metric('MCC')
    exp_tuned.remove_metric('Kappa')
    exp_tuned.get_metrics()

print(f"Experiment Type: {type(exp_tuned)}") # check the type of exp

Experiment loaded from storage.
Experiment Type: <class 'pycaret.classification.oop.ClassificationExperiment'>


In [ ]:
# Use previous created base models (with a pre-defined hyper-parameter set) to tune them with a pre-defined hyper-parameter seach space

# basemodels = exp_base.compare_models(sort='F1_Macro') # define base models

tuned_models = []
for i in base_models:
    print(f"##### Model Algorithm: {i.__class__} #####")
    tuned_model = exp_tuned.tune_model(estimator=i, optimize='F1_Macro', search_library='scikit-learn', search_algorithm='random', n_iter=20)
    tuned_models.append(tuned_model)  # Append the tuned model to the list
    print("\n")

##### Model Algorithm: <class 'sklearn.tree._classes.DecisionTreeClassifier'> #####


,Accuracy,AUC,Recall,Prec.,F1,F1_Macro
Fold,,,,,,
0,0.8430,0.9287,0.8430,0.9759,0.9011,0.7755
1,0.8492,0.9324,0.8492,0.9741,0.9032,0.7313
2,0.8336,0.9276,0.8336,0.9680,0.8912,0.6886
3,0.8402,0.9286,0.8402,0.9744,0.8986,0.7006
4,0.8252,0.9209,0.8252,0.9599,0.8826,0.6931
Mean,0.8382,0.9276,0.8382,0.9704,0.8953,0.7178
Std,0.0082,0.0038,0.0082,0.0059,0.0076,0.0325


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 20 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


##### Model Algorithm: <class 'xgboost.sklearn.XGBClassifier'> #####


...

Fitting 5 folds for each of 20 candidates, totalling 100 fits




##### Model Algorithm: <class 'sklearn.ensemble._gb.GradientBoostingClassifier'> #####


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,02:50:39
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Results
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
# Create the directory if it doesn't exist
os.makedirs(config['pycaret_exp_storage_directory'], exist_ok=True)
exp_filename = "exp_tuned.pkl"
exp_tuned.save_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")

## PyCaret Best Model Evaluation

In [ ]:
try:
    exp_filename = "exp_tuned.pkl"
    try:
        # Check if exp_tuned is defined and assign it only if it exists. "or" statment in case exp_base is falsy.
        exp_tuned = exp_tuned or ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
    except NameError:
        # If exp_tuned is not defined, assign it by loading the experiment.
        exp_tuned = ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"The experiment {exp_filename} does not exist and could not be loaded.")

In [ ]:
leaderboard_tuned = exp_tuned.get_leaderboard()
leaderboard_tuned.sort_values(by='F1_Macro', ascending=False)

In [ ]:
# returns best model based on the defined metric in the given pycaret experiment
best_model = exp_tuned.automl(optimize='F1_Macro')

# predict on test set
holdout_pred = exp_tuned.predict_model(best_model)

# show predictions df
# holdout_pred.head()

# print classification report for holdout test data
print(classification_report(holdout_pred['packaging_category'], holdout_pred['prediction_label']))
# Store f1_macro
report = classification_report(holdout_pred['packaging_category'], holdout_pred['prediction_label'], output_dict=True)
f1_score = report['accuracy']
f1_macro = report['macro avg']['f1-score']

# PyCaret AutoML: custom pre-processing; unrestricted selection of models including HPO

## Define features and target, performe oversampling, split data into train and test

In [ ]:
# Define features and target
X = df_sub.iloc[:, :-1]
y = df_sub.iloc[:, -1]  # the last column is the target

# Oversamlping
distribution_classes = y.value_counts()
print('Class distribution before oversmapling')
print(distribution_classes.to_dict())
# NOTE: Oversampling so each class has at least 100 sample; to properly apply CV and evaluation
dict_oversmapling = {
    'Metal Cassette': 100,
    'Carton tube with or w/o': 100,
    'Wooden box': 100,
    'Fabric packaging': 100,
    'Book packaging': 100
}
# define oversampling strategy
oversampler = RandomOverSampler(sampling_strategy=dict_oversmapling, random_state=SEED)
# fit and apply the transform
X_oversample, y_oversample = oversampler.fit_resample(X, y)
# Generate data set for PyCaret
df_sub_oversampled = pd.concat([X_oversample, y_oversample], axis=1)
print('Class distribution after oversmapling')
print(y_oversample.value_counts().to_dict())

## PyCaret Base Models Training Pipeline

In [ ]:
# init the ClassificationExperiment class
exp_base_custom = ClassificationExperiment()

print(f"Experiment Type: {type(exp_base_custom)}") # check the type of exp

In [ ]:
# init setup on exp
exp_base_custom.setup(
    df_sub_oversampled,
    target='packaging_category',
    train_size=0.8,
    fold=5,
    fold_strategy='stratifiedkfold',
    session_id=42
)
# add sklearn f1_score macro average
exp_base_custom.add_metric(id='f1_macro', name='F1_Macro', score_func=utils.f1_score_macro, greater_is_better=True)
exp_base_custom.remove_metric('MCC')
exp_base_custom.remove_metric('Kappa')
exp_base_custom.get_metrics()

In [ ]:
# train and compare base models

# #NOTE: class_weights are not yet supported in compare_models
# class_weights = class_weight.compute_class_weight(
#     class_weight="balanced",
#     classes=np.unique(df_sub.iloc[:, -1]),
#     y=df_sub.iloc[:, -1]
# )
# class_weight_dict = dict(enumerate(class_weights))
# basemodels = exp_base.compare_models(include=['dt', 'rf'], sort='F1_Macro', fit_kwargs={'class_weight': class_weight_dict})
# lr_clf = exp_base.create_model('lr', class_weight=class_weight_dict)

base_models = exp_base_custom.compare_models(sort='F1_Macro', n_select=exp_base_custom.models().shape[0])

In [ ]:
# Create the directory if it doesn't exist
os.makedirs(config['pycaret_exp_storage_directory'], exist_ok=True)
exp_filename = "exp_base_custom.pkl"
exp_base_custom.save_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")

## PyCaret Best Model Evaluation

In [ ]:
try:
    exp_filename = "exp_base_custom.pkl"
    try:
        # Check if exp_base_custom is defined and assign it only if it exists. "or" statment in case exp_base is falsy.
        exp_base_custom = exp_base_custom or ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
    except NameError:
        # If exp_base_custom is not defined, assign it by loading the experiment
        exp_base_custom = ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"The experiment {exp_filename} does not exist and could not be loaded.")

In [ ]:
leaderboard_base_custom = exp_base_custom.get_leaderboard()
leaderboard_base_custom.sort_values(by='F1_Macro', ascending=False)

## PyCaret Tuned Models Training Pipeline

In [ ]:
try:
    exp_filename = "exp_base_custom.pkl"
    try:
        # Check if exp_tuned_custom is defined and assign it only if it exists. "or" statment in case exp_base is falsy.
        exp_tuned_custom = exp_base_custom or ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
        print("Experiment loaded from storage.")
    except NameError:
        # If exp_tuned_custom is not defined, assign it by loading the experiment.
        exp_tuned_custom = exp_base_custom or ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"The experiment {exp_filename} does not exist and could not be loaded.")
    print("Define new pycaret experiment for tuning.")
    exp_tuned_custom = ClassificationExperiment()
    exp_tuned_custom.setup(
        df_sub_oversampled,
        target='packaging_category',
        train_size=0.8,
        fold=5,
        fold_strategy='stratifiedkfold',
        session_id=42
    )
    # add sklearn f1_score macro average
    exp_tuned_custom.add_metric(id='f1_macro', name='F1_Macro', score_func=utils.f1_score_macro, greater_is_better=True)
    exp_tuned_custom.remove_metric('MCC')
    exp_tuned_custom.remove_metric('Kappa')
    exp_tuned_custom.get_metrics()

print(f"Experiment Type: {type(exp_tuned_custom)}") # check the type of exp

In [ ]:
# Use previous created base models (with a pre-defined hyper-parameter set) to tune them with a pre-defined hyper-parameter seach space

# basemodels = exp_base.compare_models(sort='F1_Macro') # define base models

tuned_models = []
for i in base_models:
    print(f"##### Model Algorithm: {i.__class__} #####")
    tuned_model = exp_tuned_custom.tune_model(estimator=i, optimize='F1_Macro', search_library='scikit-learn', search_algorithm='random', n_iter=20)
    tuned_models.append(tuned_model)  # Append the tuned model to the list
    print("\n")

In [ ]:
# Create the directory if it doesn't exist
os.makedirs(config['pycaret_exp_storage_directory'], exist_ok=True)
exp_filename = "exp_tuned_custom.pkl"
exp_tuned_custom.save_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")

## PyCaret Best Model Evaluation

In [ ]:
try:
    exp_filename = "exp_tuned_custom.pkl"
    try:
        # Check if exp_tuned_custom is defined and assign it only if it exists. "or" statment in case exp_base is falsy.
        exp_tuned_custom = exp_tuned_custom or ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
    except NameError:
        # If exp_tuned_custom is not defined, assign it by loading the experiment
        exp_tuned_custom = ClassificationExperiment.load_experiment(f"{config['pycaret_exp_storage_directory']}/{exp_filename}")
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"The experiment {exp_filename} does not exist and could not be loaded.")

leaderboard_tuned_custom = exp_tuned_custom.get_leaderboard()
leaderboard_tuned_custom.sort_values(by='F1_Macro', ascending=False)

In [ ]:
# returns best model based on the defined metric in the given pycaret experiment
best_model = exp_tuned_custom.automl(optimize='F1_Macro')

# predict on test set
holdout_pred = exp_tuned_custom.predict_model(best_model)

# show predictions df
# holdout_pred.head()

# print classification report for holdout test data
print(classification_report(holdout_pred['packaging_category'], holdout_pred['prediction_label']))
report = classification_report(holdout_pred['packaging_category'], holdout_pred['prediction_label'], output_dict=True)
f1_score = report['accuracy']
f1_macro = report['macro avg']['f1-score']

## Track performance using MLflow

In [ ]:
# NOTE: Change to a meaningful name
EXPERIMENT_NAME = "AutoPackagingCategories"
RUN_NAME = "run_AutoML_PyCaret"

with open('../env_vars.yml', 'r') as file:
    env_vars = yaml.safe_load(file)

project_dir = env_vars['project_directory']
os.makedirs(project_dir + '/mlruns', exist_ok=True)

mlflow.set_tracking_uri("file://" + project_dir + "/mlruns")

try:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    EXPERIMENT_ID = experiment.experiment_id
except AttributeError:
    EXPERIMENT_ID = mlflow.create_experiment(
        EXPERIMENT_NAME,
        # mlflow.set_artifact_uri("file://" + project_dir + "/artifacts/")
    )

current_time = datetime.datetime.now()
time_stamp = str(current_time)
# NOTE: Change to a meaningful name for the single trial
# exp_run_name = f"run_MeaningfulTrialName_{time_stamp}"
exp_run_name = f"{RUN_NAME}_{time_stamp}"

# Start MLflow
with mlflow.start_run(experiment_id=EXPERIMENT_ID, run_name=exp_run_name) as run:

    # Retrieve run id
    RUN_ID = run.info.run_id

    # Track parameters
    # track pipeline configs: preprocessing_pipeline
    mlflow.log_dict(exp_tuned.get_config('pipeline').named_steps, "preprocessing_pipeline.json")

    # mode specfic parameters
    mlflow.log_param('model', type(best_model))
    mlflow.log_param('model_configs', best_model.get_params())

    # Track metrics
    mlflow.log_dict(report, "classification_report.json")
    mlflow.log_metric("Report_Test_f1_score", f1_score)
    mlflow.log_metric("Report_Test_f1_macro", f1_macro)

    # Track model
    # mlflow.sklearn.log_model(clf, "classifier")

In [ ]:
import time

def keep_alive_with_cpu_activity(duration_hours=1):
    """
    Keeps the compute instance alive by running a periodic CPU task for a specified duration.
    """
    start_time = time.time()
    end_time = start_time + duration_hours * 3600  # convert hours to seconds
    print(f"Keeping the instance alive for {duration_hours} hours with periodic CPU activity.")
    print("To stop the function, create an empty file named stop_signal.txt in the same directory as your notebook.")
    print(f"os.path: {os.path}")

    while time.time() < end_time:
        # Check if stop signal file exists
        if os.path.isfile("stop_signal.txt"):
            print("Stop signal received. Exiting the loop.")
            break

        # Perform a small computation to generate CPU activity
        _ = np.random.rand(1000000, 1000000).dot(np.random.rand(1000000, 1000000))
        time.sleep(60)  # Sleep for 60 seconds

    print("Finished keeping the instance alive.")

# Run for the desired duration (e.g., 4 hours)
keep_alive_with_cpu_activity(duration_hours=8)